### In this notebook I will look at the full Bdot measurements of the 25cm plane.

I will also check if taking the curl of the magnetic field recovers a current centered at the center of the circulation in the b field.  I will test schemes for locating the current centroid.

## Imports

In [1]:
import numpy as np
import scipy.optimize as opti
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.tri import Triangulation, LinearTriInterpolator
from scipy.optimize import curve_fit
from scipy import odr
from scipy.optimize import leastsq
from skimage.feature import peak_local_max
import scipy.ndimage as ndimage
import cPickle as pickle
from scipy.integrate import odeint

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import sys
sys.path.append('/Users/vonderlinden2/rsx_analysis/writing_to_vtk/source')
import structured_3d_vtk as struc_3d

ImportError: No module named structured_3d_vtk

## Define convience function and load magnetic field data.

In [4]:
def read_and_return_vector_and_grid(file_path, quantity_name, grid_name):
    r"""
    """
    array_dict = np.load(file_path)
    quantity = array_dict[quantity_name]
    grid = array_dict[grid_name]
    return quantity, grid

In [5]:
file_path = '../../output/centroid_fitting/2016-08-12/B_z0249_'
ending = '.npz' 
b_fields = []
for time_point in xrange(21):
    name = file_path + str(time_point).zfill(4) + ending
    b_field, grid = read_and_return_vector_and_grid(name, 'b', 'grid')
    b_fields.append(b_field)

In [6]:
grid[1][-1]

array([-0.043     , -0.04198765, -0.04097531, -0.03996296, -0.03895062,
       -0.03793827, -0.03692593, -0.03591358, -0.03490123, -0.03388889,
       -0.03287654, -0.0318642 , -0.03085185, -0.02983951, -0.02882716,
       -0.02781481, -0.02680247, -0.02579012, -0.02477778, -0.02376543,
       -0.02275309, -0.02174074, -0.0207284 , -0.01971605, -0.0187037 ,
       -0.01769136, -0.01667901, -0.01566667, -0.01465432, -0.01364198,
       -0.01262963, -0.01161728, -0.01060494, -0.00959259, -0.00858025,
       -0.0075679 , -0.00655556, -0.00554321, -0.00453086, -0.00351852,
       -0.00250617, -0.00149383, -0.00048148,  0.00053086,  0.00154321,
        0.00255556,  0.0035679 ,  0.00458025,  0.00559259,  0.00660494,
        0.00761728,  0.00862963,  0.00964198,  0.01065432,  0.01166667,
        0.01267901,  0.01369136,  0.0147037 ,  0.01571605,  0.0167284 ,
        0.01774074,  0.01875309,  0.01976543,  0.02077778,  0.02179012,
        0.02280247,  0.02381481,  0.02482716,  0.02583951,  0.02

# Method 2: Integrate field lines off the max and fit circle to the field lines

## Try it out by hand

In [7]:
def to_min(params, points):
    a = 2.*params[0]
    b = 2.*params[1]
    c = params[2]**2 - params[1]**2 - params[0]**2
    return a*points[0] + b*points[1] + c - points[0]**2 - points[1]**2

def calculate_radius(x, y, x_0, y_0):
    r"""
    Calculate radius.
    """
    return (x - x_0)**2 + (y - y_0)**2


def estimate_radius(data, x_0, y_0):
    r"""
    Estimate radius from data.
    """
    return calculate_radius(data[:, 0], data[:, 1], x_0, y_0).mean()

params_guess = [0, 0, 0.01]

In [11]:
def d_l(l, t, interpolator_x, interpolator_y):
    return np.asarray([interpolator_x([l[0], l[1]])[0], 
                       interpolator_y([l[0], l[1]])[0]])

def find_centroid(grid, bx_interpolator, by_interpolator, 
                  distance_thres=0.001, filter_size=5,
                  integration_length=10, integration_steps=100,
                  launch_point_step_factor=0.1, max_count=50):
    r"""
    """
    b_fields_x = bx_interpolator(grid[0][:, :], grid[1][:, :])
    b_fields_y = by_interpolator(grid[0][:, :], grid[1][:, :])
    b_fields = [b_fields_x, b_fields_y]
    x_min, x_max = grid[0].min(), grid[0].max()
    y_min, y_max = grid[1].min(), grid[1].max()
    magnitude = np.sqrt(b_fields[0][:, :]**2 + b_fields[1][:, :]**2)
    filtered_magnitude = ndimage.gaussian_filter(magnitude, filter_size)
    max_index = np.unravel_index(filtered_magnitude.argmax(), 
                                 filtered_magnitude.shape)
    
    center_points = []
    radii = []
    center_points = []
    streamlines = []
    direction = [0, 0]
    distance = 100
    launch_point = (grid[0][max_index], grid[1][max_index])
    count = 0
    while distance >= distance_thres:
        #print 'launch', launch_point
        #print distance
        t2 = np.linspace(0, integration_length, integration_steps)
        t1 = np.linspace(0, -integration_length, integration_steps)
        stream2 = odeint(d_l, launch_point, t2, args=(bx_interpolator, by_interpolator))
        stream1 = odeint(d_l, launch_point, t1, args=(bx_interpolator, by_interpolator))
        print 'stream', stream1, stream2
        streamline = np.concatenate((stream1, stream2))
        size = streamline[np.invert(np.isnan(streamline))].size
        streamline = streamline[np.invert(np.isnan(streamline))].reshape(int(size/2.), 2)
        
        circle_params, success = leastsq(to_min, params_guess, args=np.asarray([streamline[:, 0], streamline[:, 1]]))
        direction = [circle_params[0] - launch_point[0], circle_params[1] - launch_point[1]]
        distance = np.sqrt(direction[0]**2. + direction[1]**2.)
        center_point = (circle_params[0], circle_params[1])
        launch_point = [launch_point[0] + direction[0] * launch_point_step_factor,
                        launch_point[1] + direction[1] * launch_point_step_factor]
        center_points.append(center_point)
        #print 'center', center_point
        radii.append(circle_params[0])
        streamlines.append(streamline)
        if (launch_point[0] <= x_min or
            launch_point[0] >= x_max or
            launch_point[1] <= y_min or 
            launch_point[1] >= y_max or
            count > max_count):
            break
        count += 1
    centroid = center_point
    return centroid, center_points, radii, streamlines, max_index

In [12]:
centroids = []
for time_point in xrange(1):
    print time_point
    bx_interpolator = pickle.load(open('../../output/centroid_fitting/2016-08-12/B_z0249_x_' + str(time_point).zfill(4) + '.p', 'rb'))
    by_interpolator = pickle.load(open('../../output/centroid_fitting/2016-08-12/B_z0249_y_' + str(time_point).zfill(4) + '.p', 'rb'))

    (centroid, center_points, 
     radii, streamlines, 
     max_index) = find_centroid(grid, 
                                bx_interpolator, 
                                by_interpolator,
                                launch_point_step_factor=0.05,
                                integration_length=20)

    centroids.append(centroid)

0
stream [[  4.00000000e-03   8.62962963e-03]
 [  3.67592958e-03   9.03575048e-03]
 [  3.36077889e-03   9.42622384e-03]
 [  3.04273356e-03   9.80181286e-03]
 [  2.71095807e-03   1.01691870e-02]
 [  2.36550342e-03   1.05315642e-02]
 [  2.00639694e-03   1.08886352e-02]
 [  1.63930014e-03   1.12389212e-02]
 [  1.28116070e-03   1.15863469e-02]
 [  9.33689902e-04   1.19499104e-02]
 [  6.00370242e-04   1.22977588e-02]
 [  2.81527549e-04   1.26081974e-02]
 [ -2.43140802e-05   1.28833640e-02]
 [ -3.18543937e-04   1.31252177e-02]
 [ -6.10351247e-04   1.33464321e-02]
 [ -9.17544577e-04   1.35709546e-02]
 [ -1.24135758e-03   1.37992832e-02]
 [ -1.58227429e-03   1.40309232e-02]
 [ -1.92656231e-03   1.42557022e-02]
 [ -2.26624143e-03   1.44677157e-02]
 [ -2.60210263e-03   1.46671564e-02]
 [ -2.93493751e-03   1.48550961e-02]
 [ -3.26534068e-03   1.50357963e-02]
 [ -3.59373549e-03   1.52098104e-02]
 [ -3.92055537e-03   1.53771108e-02]
 [ -4.24072420e-03   1.55943718e-02]
 [ -4.54171679e-03   1.582041

In [18]:
from datetime import datetime
import os

today = datetime.today()
today = today.strftime('%Y-%m-%d')

out_dir = '../output/' + today
try:
    os.makedirs(out_dir)
except:
    pass

In [19]:
centroids = np.asarray(centroids)
np.savetxt(out_dir + '/field_nulls.txt', centroids, header='magnetic field null positions in the z=0.249m plane,' + 
           'determined by fitting circles to integrated field lines starting at max magnitude and moving succesive' +
           'towards the center of circles.')

In [37]:
from scipy.integrate import dblquad

In [51]:
def integrate_flux(centroid, radius, bz_interpolator, limits):
    if (centroid[0] - radius < limits[0] or centroid[0] + radius > limits[1] or 
        centroid[1] - radius < limits[2] or centroid[1] + radius > limits[3]):
        return -1
    gfun = lambda x: -np.sqrt(radius**2 - (x-centroid[0])**2)
    hfun = lambda x: np.sqrt(radius**2 - (x-centroid[0])**2)
    bz_interpolator_bias  = lambda x, y: bz_interpolator(x, y) + 0.02
    return dblquad(bz_interpolator_bias, centroid[0] - radius, centroid[0] + radius, gfun, hfun)

In [52]:
bz_interpolator = pickle.load(open('../output/2016-08-12/B_z0249_z_' + str(0).zfill(4) + '.p', 'rb'))
bz_interpolator(centroids[0])

array([-0.00137151])

In [53]:
for time_point in xrange(250):
    bz_interpolator = pickle.load(open('../output/2016-08-12/B_z0249_z_' + str(time_point).zfill(4) + '.p', 'rb'))
    print integrate_flux(centroids[time_point], 0.001, bz_interpolator, 
                         (-0.028, 0.025, -0.043, 0.039))

(5.839913639626571e-08, 2.8013188400032477e-09)
(5.794193299256448e-08, 3.0422770069225602e-09)
(5.7735296762456624e-08, 2.4717155332413957e-09)
(5.7502258847160966e-08, 2.9614017336170323e-09)
(5.738798950942609e-08, 2.6281532307919568e-09)
(5.7277990692961765e-08, 2.5392591859959736e-09)
(5.7756166082988825e-08, 3.050361493343241e-09)
(5.802438215116667e-08, 3.0494530812849156e-09)
(5.817519681831278e-08, 3.0718529972046664e-09)
(5.8279023778263414e-08, 3.0779759471455983e-09)
(5.7958835321432546e-08, 2.9449962371746143e-09)
(5.865219555129318e-08, 3.055036944552988e-09)
(5.76783269957705e-08, 3.038030956076129e-09)
(5.769966055840536e-08, 3.0524209784619142e-09)
(5.7209417551959144e-08, 3.0249871221650753e-09)
(5.698898810635763e-08, 3.014345552082165e-09)
(5.7041524513158006e-08, 3.0158685181121404e-09)
(5.7280107268094833e-08, 3.0488159117304395e-09)
(5.8049195793130654e-08, 3.0859397534695927e-09)
(5.866461968581144e-08, 3.1205410192632266e-09)
(5.892174418037519e-08, 3.041036054